The purpose of this section is to convert the reaction IDs and metabolite IDs to the human readable BIGG format. For metabolites and reactions which do not appear in kegg, new bigg-like IDs will be created. In addition, the subsystems IDs in iAT601 are repeated but with small variations (e.g. captilization). Unfortunately at the time there is no standard for subsytem nomenclature. Certain metabolites appear in both isomeric forms, which leads to repeated reactions and pathway connectivity issues ( This may not be a nomenclature problem).

## Import model

In [ ]:
import cobra as cb
import os

In [2]:
cbmodel = cb.io.read_sbml_model(os.path.abspath("iAT601/iAT601_CB.xml"))
cbmodel = cb.io.read_sbml_model(os.path.abspath("iAT601/iAT601_CE.xml"))

cobra\core\reaction.py:394 UserWarning: malformed gene_reaction_rule 'Clo1313_1355; Clo1313_1353; Clo1313_1356' for <Reaction R01197_c at 0x212786ba470>
cobra\core\reaction.py:394 UserWarning: malformed gene_reaction_rule 'Clo1313_2018; Clo1313_2017' for <Reaction R03660_c at 0x212787d7898>


cobra\core\reaction.py:394 UserWarning: malformed gene_reaction_rule 'Clo1313_1355; Clo1313_1353; Clo1313_1356' for <Reaction R01197_c at 0x21278d83a20>
cobra\core\reaction.py:394 UserWarning: malformed gene_reaction_rule 'Clo1313_2018; Clo1313_2017' for <Reaction R03660_c at 0x21278e92e48>


Like the warning gracefully indicates, the GPR is incorrect, we will replace those with an "or" for the sake of generality and later revisit them for accuracy. 

In [3]:
badGPR = ['Clo1313_1355; Clo1313_1353; Clo1313_1356','Clo1313_2018; Clo1313_2017']
goodGPR = ['Clo1313_1355 or Clo1313_1353 or Clo1313_1356','Clo1313_2018 or Clo1313_2017']

def fixGPR(fin,fout):
    for line in fin:
        if badGPR[0] in line:
            fout.write(line.replace(badGPR[0],goodGPR[0]))
        elif badGPR[1] in line:
            fout.write(line.replace(badGPR[1],goodGPR[1]))
        else:
            fout.write(line)
    fin.close()
    fout.close()
    
fin_CB = open(os.path.abspath("iAT601/iAT601_CB.xml"), 'r')
fout_CB = open(os.path.abspath("iAT601/iAT601_CB_fixed_GPR.xml"), 'w')
fixGPR(fin_CB,fout_CB)

fin_CE = open(os.path.abspath("iAT601/iAT601_CE.xml"), 'r')
fout_CE = open(os.path.abspath("iAT601/iAT601_CE_fixed_GPR.xml"), 'w')
fixGPR(fin_CE,fout_CE)

In [4]:
cbmodel = cb.io.read_sbml_model(os.path.abspath("iAT601/iAT601_CB_fixed_GPR.xml"))
cemodel = cb.io.read_sbml_model(os.path.abspath("iAT601/iAT601_CE_fixed_GPR.xml"))

What is the actual difference between the two models?

In [5]:
# Compare reaction and metabolite ids
print(set([reaction.id for reaction in cbmodel.reactions]) == set([reaction.id for reaction in cemodel.reactions]))
print(set([metabolite.id for metabolite in cbmodel.metabolites]) == set([metabolite.id for metabolite in cemodel.metabolites]))

True
True


In [6]:
# Compare reaction bounds
with open(os.path.abspath(os.path.join('iAT601','cb_ce_diff.tsv')), 'w') as myfile:
    myfile.write('rxn_id\tcb_bounds\tce_bounds\n')
    for cb_reaction in cbmodel.reactions:
        for ce_reaction in cemodel.reactions:
            if cb_reaction.id == ce_reaction.id:
                if cb_reaction.bounds != ce_reaction.bounds:
                    print('Rxn:',cb_reaction.id,'cb_bounds:',cb_reaction.bounds,'ce_bounds',ce_reaction.bounds)
                    myfile.write('{0}\t{1}\t{2}\n'.format(cb_reaction.id,str(cb_reaction.bounds),str(ce_reaction.bounds)))

Rxn: EXC_IN_m20 cb_bounds: (3.42, 3.48) ce_bounds (0.0, 0.0)
Rxn: EXC_IN_m97 cb_bounds: (0.0, 0.0) ce_bounds (1.0517, 1.078)
Rxn: EXC_OUT_m51 cb_bounds: (4.09, 4.29) ce_bounds (2.6, 2.66)
Rxn: EXC_OUT_m52 cb_bounds: (1.77, 3.49) ce_bounds (3.25, 3.55)
Rxn: EXC_OUT_m53 cb_bounds: (1.76, 1.78) ce_bounds (1.38, 1.39)
Rxn: EXC_OUT_m54 cb_bounds: (0.0, 0.2) ce_bounds (0.0, 0.0)
Rxn: EXC_OUT_m55 cb_bounds: (7.48, 8.24) ce_bounds (0.0, 1000.0)
Rxn: EXC_OUT_m56 cb_bounds: (0.0, 0.2) ce_bounds (0.0, 1000.0)


In [7]:
# write out lb and ub for media
print(cbmodel.metabolites.m20.name)
print(cbmodel.metabolites.m97.name)
print(cbmodel.medium)
print(cemodel.medium)
with open(os.path.abspath(os.path.join('iAT601','media', 'iAT601_cb_medium.csv')), 'w') as myfile:
    myfile.write('rxn_id,rxn_lb,rxn_ub\n')
    for key, value in cbmodel.medium.items():
        rxn = getattr(cbmodel.reactions, key)
        myfile.write("{0},{1},{2}\n".format(key, rxn.lower_bound, rxn.upper_bound))
        
with open(os.path.abspath(os.path.join('iAT601','media', 'iAT601_ce_medium.csv')), 'w') as myfile:
    myfile.write('rxn_id,rxn_lb,rxn_ub\n')
    for key, value in cemodel.medium.items():
        rxn = getattr(cemodel.reactions, key)
        myfile.write("{0},{1},{2}\n".format(key, rxn.lower_bound, rxn.upper_bound))

Cellobiose_C12H22O11
Cellohexaose
{'EXC_IN_m20': 3.48, 'EXC_IN_m17': 5.0, 'EXC_IN_m21': 5.0, 'EXC_IN_m22': 1000.0, 'EXC_IN_m23': 1000.0, 'EXC_IN_m24': 1000.0, 'EXC_IN_m25': 1000.0, 'EXC_IN_m26': 1000.0, 'EXC_IN_m27': 1000.0, 'EXC_IN_m28': 1000.0, 'EXC_IN_m30': 1000.0, 'EXC_IN_m101': 1000.0, 'EXC_BOTH_m18': 1000.0, 'EXC_BOTH_m35': 0.5}
{'EXC_IN_m17': 5.0, 'EXC_IN_m21': 5.0, 'EXC_IN_m22': 1000.0, 'EXC_IN_m23': 1000.0, 'EXC_IN_m24': 1000.0, 'EXC_IN_m25': 1000.0, 'EXC_IN_m26': 1000.0, 'EXC_IN_m27': 1000.0, 'EXC_IN_m28': 1000.0, 'EXC_IN_m30': 1000.0, 'EXC_IN_m97': 1.078, 'EXC_IN_m101': 1000.0, 'EXC_BOTH_m18': 1000.0, 'EXC_BOTH_m35': 0.5}


The difference is just in the media. We will continue with the cellulose model from now on, and when refering to specific simulation we will adjust to the appropriate media.

In [8]:
model = cbmodel.copy()
model.id = 'iSG601_1'

## Further model cleaning

Remove metabolites that do not participate in any reaction, as well as reactions without assigned metabolites. ( Note cb.manipulation.delete.prune_unused_metabolites(model)) is not working correctly, it needs to be run several times)

In [9]:
junk_met_id = []
for met in model.metabolites:
    if not met.reactions:
        junk_met_id.append(met.id)

print(len(junk_met_id))

with open(os.path.abspath(os.path.join('iAT601','unused_metabolites_in_iAT601.csv')), 'w') as myfile:
    for item in junk_met_id:
        myfile.write("%s\n" % item)


51


In [10]:
# Check unused reactions
cb.manipulation.delete.prune_unused_reactions(model)

[]

In [11]:
#The reactions R02948_c R02948b_c are repeated..
print(model.reactions.R02948_c.metabolites)
print(model.reactions.R02948b_c.metabolites)

print(model.reactions.R02948_c.gene_reaction_rule)
print(model.reactions.R02948b_c.gene_reaction_rule)

#model.remove_reactions('R02948b_c')
 # Remove copy
model.remove_reactions(['R02948b_c'])

{<Metabolite C06010_c at 0x2127a855e48>: -1.0, <Metabolite C00810_c at 0x2127a832ba8>: 1.0, <Metabolite C00011_c at 0x2127a6a5dd8>: 1.0}
{<Metabolite C00900_c at 0x2127a832eb8>: -1.0, <Metabolite C00810_c at 0x2127a832ba8>: 1.0, <Metabolite C00011_c at 0x2127a6a5dd8>: 1.0}




In [12]:
cb.io.mat.save_matlab_model(model, os.path.abspath(os.path.join('iSG', 'iSG601_1.mat'))) 
cb.io.sbml.write_cobra_model_to_sbml_file(model,os.path.abspath(os.path.join('iSG','iSG601_1.xml'))) 

For transport and exchange reactions we will replace the old metabolite ids: